<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/Solar-LLM-ZeroToAll/02_prompt_engineering.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
! pip3 install -qU langchain-upstage getpass4

In [2]:
import os
import getpass
import warnings
warnings.filterwarnings("ignore")

UPSTAGE_API_KEY = getpass.getpass('Enter your API Key')
_ = os.environ.setdefault("UPSTAGE_API_KEY", UPSTAGE_API_KEY)

In [3]:
# Quick hello world
from langchain_upstage import ChatUpstage

llm = ChatUpstage()
llm.invoke("What is the capital of France")

AIMessage(content='The capital of France is Paris.', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 16, 'total_tokens': 24}, 'model_name': 'solar-1-mini-chat-240612', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-100a7ee2-94db-4af9-9d50-01fff6b4d397-0', usage_metadata={'input_tokens': 16, 'output_tokens': 8, 'total_tokens': 24})

In [4]:
# Chat prompt
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human", "What is the capital of France?"),
        ("ai", "I know of it. It's Paris!!"),
        ("human", "What about Korea?"),
    ]
)

# 3. define chain
from langchain_core.output_parsers import StrOutputParser

chain = chat_prompt | llm | StrOutputParser()
chain.invoke({})



"Oh, that's a tricky one. I'm not sure if I know the answer to that. Let me think... Oh wait, I remember now! The capital of Korea is Seoul."

In [5]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
Q: The cafeteria had 23 apples. 
If they used 20 to make lunch and bought 6 more, 
how many apples do they have?

A: the answer is
"""
)
chain = prompt_template | llm | StrOutputParser()
chain.invoke({})

'The cafeteria started with 23 apples. They used 20 of them, so they had 23 - 20 = 3 apples left. Then they bought 6 more apples, so they had 3 + 6 = 9 apples. The final answer is \\boxed{9}.'

In [6]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?

A: The answer is 11.

Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?

A: the answer is
"""
)
chain = prompt_template | llm | StrOutputParser()
chain.invoke({})

'The answer is 29.'

![CoT](figures/cot.webp)

from https://arxiv.org/abs/2201.11903

In [7]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?

A: Roger started with 5 balls. 2 cans of 3 tennis balls
each is 6 tennis balls. 5 + 6 = 11. The answer is 11.

Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?"""
)
chain = prompt_template | llm | StrOutputParser()
chain.invoke({})

'The cafeteria started with 23 apples. They used 20, so they had 23 - 20 = 3 apples left. Then they bought 6 more apples, so they had 3 + 6 = 9 apples. The answer is 9.'

![Zero-Shot COT](figures/zero-cot.webp)

From https://arxiv.org/abs/2205.11916

In [8]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?

A: The answer is 11.

Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?

A: Let's think step by step.
"""
)
chain = prompt_template | llm | StrOutputParser()
chain.invoke({})

'Step 1: The cafeteria started with 23 apples.\nStep 2: They used 20 apples to make lunch.\nStep 3: After using the apples, they had 23 - 20 = 3 apples left.\nStep 4: They bought 6 more apples.\nStep 5: To find the total number of apples, we add the remaining apples to the apples they bought.\nStep 6: So, the cafeteria has 3 + 6 = 9 apples now.\n\nThe answer is 9.'

## divide and conquer
![divideandconquer](figures/divideandconquer.png)

In [9]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Please provide three questions from the following text:
    ---
    We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, 
    demonstrating superior performance in various natural language processing (NLP) tasks. 
    Inspired by recent efforts to efficiently up-scale LLMs, 
    we present a method for scaling LLMs called depth up-scaling (DUS), 
    which encompasses depthwise scaling and continued pretraining.
    In contrast to other LLM up-scaling methods that use mixture-of-experts, 
    DUS does not require complex changes to train and inference efficiently. 
    We show experimentally that DUS is simple yet effective 
    in scaling up high-performance LLMs from small ones. 
    Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, 
    a variant fine-tuned for instruction-following capabilities, 
    surpassing Mixtral-8x7B-Instruct. 
    SOLAR 10.7B is publicly available under the Apache 2.0 license, 
    promoting broad access and application in the LLM field.
    """
)
chain = prompt_template | llm | StrOutputParser()
chain.invoke({})

'1. What is SOLAR 10.7B and what are its key features?\n2. How does DUS differ from other LLM up-scaling methods, and what are its advantages?\n3. How does SOLAR 10.7B-Instruct compare to Mixtral-8x7B-Instruct in terms of instruction-following capabilities?'

In [10]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Please extract three keywords from the following text:
    ---
    We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, 
    demonstrating superior performance in various natural language processing (NLP) tasks. 
    Inspired by recent efforts to efficiently up-scale LLMs, 
    we present a method for scaling LLMs called depth up-scaling (DUS), 
    which encompasses depthwise scaling and continued pretraining.
    In contrast to other LLM up-scaling methods that use mixture-of-experts, 
    DUS does not require complex changes to train and inference efficiently. 
    We show experimentally that DUS is simple yet effective 
    in scaling up high-performance LLMs from small ones. 
    Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, 
    a variant fine-tuned for instruction-following capabilities, 
    surpassing Mixtral-8x7B-Instruct. 
    SOLAR 10.7B is publicly available under the Apache 2.0 license, 
    promoting broad access and application in the LLM field.
    """
)
chain = prompt_template | llm | StrOutputParser()
chain.invoke({})

'1. SOLAR 10.7B\n2. Large Language Model (LLM)\n3. Depth Up-Scaling (DUS)'

In [11]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Please provide one question from the following text 
    regarding "Depth up-scaling (DUS)":
    
    ---
    We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, 
    demonstrating superior performance in various natural language processing (NLP) tasks. 
    Inspired by recent efforts to efficiently up-scale LLMs, 
    we present a method for scaling LLMs called depth up-scaling (DUS), 
    which encompasses depthwise scaling and continued pretraining.
    In contrast to other LLM up-scaling methods that use mixture-of-experts, 
    DUS does not require complex changes to train and inference efficiently. 
    We show experimentally that DUS is simple yet effective 
    in scaling up high-performance LLMs from small ones. 
    Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, 
    a variant fine-tuned for instruction-following capabilities, 
    surpassing Mixtral-8x7B-Instruct. 
    SOLAR 10.7B is publicly available under the Apache 2.0 license, 
    promoting broad access and application in the LLM field.
    """
)
chain = prompt_template | llm | StrOutputParser()
chain.invoke({})

'What is the method for scaling large language models (LLMs) called, which encompasses depthwise scaling and continued pretraining, and does not require complex changes to train and inference efficiently, as introduced in the text regarding SOLAR 10.7B?'

In [12]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)
prompt_template.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.'

In [13]:
chain = prompt_template | llm | StrOutputParser()
chain.invoke({"adjective": "funny", "content": "chickens"})

'Why did the chicken cross the playground?\n\nTo get to the other slide!'

In [14]:
chain.invoke({"adjective": "funny", "content": "beef"})

'Why did the beef cross the road?\n\nTo get to the other side!\n\nBut seriously, beef is a type of meat that comes from cows. It\'s a popular ingredient in many dishes around the world. Did you know that the average American consumes about 27 kilograms of beef per year? That\'s a lot of steaks and hamburgers!\n\nBut let\'s get back to the joke. The punchline "To get to the other side!" is a play on the classic "Why did the chicken cross the road?" joke. The answer to that question is usually "To get to the other side," but the punchline is often followed by a witty comment or observation. In this case, the punchline is simply "To get to the other side," but the fact that it\'s about beef instead of a chicken makes it funny.\n\nI hope you found that joke amusing. Do you have any other questions about beef or any other topic?'

In [15]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Please provide one question from the following text 
    regarding "{keyword}":
    
    ---
    {text}
    """
)
chain = prompt_template | llm | StrOutputParser()
keyword = "DUS"
text = """
We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, 
    demonstrating superior performance in various natural language processing (NLP) tasks. 
    Inspired by recent efforts to efficiently up-scale LLMs, 
    we present a method for scaling LLMs called depth up-scaling (DUS), 
    which encompasses depthwise scaling and continued pretraining.
    In contrast to other LLM up-scaling methods that use mixture-of-experts, 
    DUS does not require complex changes to train and inference efficiently. 
    We show experimentally that DUS is simple yet effective 
    in scaling up high-performance LLMs from small ones. 
    Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, 
    a variant fine-tuned for instruction-following capabilities, 
    surpassing Mixtral-8x7B-Instruct. 
    SOLAR 10.7B is publicly available under the Apache 2.0 license, 
    promoting broad access and application in the LLM field.
"""
chain.invoke({"keyword": keyword, "text": text})

'What is the name of the large language model introduced in the text, and what is its parameter size?'

# Excercise 

Complete this code to create excellent questions from given documents using the following steps:
1. Include Few Shot examples, including CoT.
2. Generate keywords and topics first.
3. Iterate through each keyword and topic to generate questions.
4. Compare the generated questions with zero-shot generated questions.


# References
* https://platform.openai.com/docs/guides/prompt-engineering 
* https://docs.anthropic.com/claude/docs/intro-to-prompting 
* https://smith.langchain.com/hub 